In [1]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from utils import *

In [3]:
file_path='../../data/df_input_update_ori_20240812.csv'
df_input_update_ori = pd.read_csv(file_path) 
df_input = df_input_update_ori.copy()
df_input.iloc[:,0:7] = df_input.iloc[:,0:7].applymap(lambda x: np.log1p(x))
df_input=df_input.iloc[:, 0:8]

info_path = '../../data/Info.csv'
df_info = pd.read_csv(info_path)

df = df_input.copy()
df.columns = [col.replace('_Concentration (mM)', '') for col in df.columns[:-1]] + ['num_vesicles']

# Step 2: Create mapping from df_input_update_ori column names to df_info["Name"]
column_to_name = {
    'decanoic acid': 'Decanoic acid',
    'decanoate': 'Decanoate',
    'decylamine': 'Decylamine',
    'decyl trimethylamine': 'Decyltrimethyl ammonium bromid',
    'decylsulfate': 'Decyl sodium sulfate',
    'decanol': 'Decanol',
    'monocaprin': 'Glycerol monodecanoate'
}

# Step 3: Get SMILES mapping from df_info
name_to_smiles = dict(zip(df_info['Name'], df_info['SMILES']))

# Step 4: Construct new dataframe
new_data = {}

for col in df.columns[:-1]:  # Skip num_vesicles
    chem_name = column_to_name[col]
    smiles = name_to_smiles.get(chem_name, '')
    new_data[f"{col}_SMILES"] = [smiles] * len(df)
    new_data[f"{col}_Concentration"] = df[col]

# Step 5: Add num_vesicles
new_data['num_vesicles'] = df['num_vesicles']

# Step 6: Create the final dataframe
df_structured = pd.DataFrame(new_data)

# Rename columns to the desired format
new_column_names = [
    'smi1', 'conc1',
    'smi2', 'conc2',
    'smi3', 'conc3',
    'smi4', 'conc4',
    'smi5', 'conc5',
    'smi6', 'conc6',
    'smi7', 'conc7',
    'vesicles_formation'
]

# Apply the new column names
df_structured.columns = new_column_names
df_structured

/var/folders/wn/w203521n4h18b4_rz5rd9tw80000gp/T/ipykernel_29606/3573193812.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_input.iloc[:,0:7] = df_input.iloc[:,0:7].applymap(lambda x: np.log1p(x))
/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,smi1,conc1,smi2,conc2,smi3,conc3,smi4,conc4,smi5,conc5,smi6,conc6,smi7,conc7,vesicles_formation
0,CCCCCCCCCC(=O)O,0.615186,CCCCCCCCCC(=O)[O-],0.955511,CCCCCCCCCCN,1.231101,CCCCCCCCCC[N+](C)(C)C.[Br-],1.552868,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.677097,CCCCCCCCCCO,0.441476,CCCCCCCCCC(=O)OCC(CO)O,0.579418,22
1,CCCCCCCCCC(=O)O,1.487270,CCCCCCCCCC(=O)[O-],0.644482,CCCCCCCCCCN,0.175633,CCCCCCCCCC[N+](C)(C)C.[Br-],1.056053,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.216395,CCCCCCCCCCO,0.207827,CCCCCCCCCC(=O)OCC(CO)O,0.107059,9
2,CCCCCCCCCC(=O)O,1.163151,CCCCCCCCCC(=O)[O-],0.705570,CCCCCCCCCCN,1.294727,CCCCCCCCCC[N+](C)(C)C.[Br-],1.787584,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.658228,CCCCCCCCCCO,0.159138,CCCCCCCCCC(=O)OCC(CO)O,0.202941,6
3,CCCCCCCCCC(=O)O,1.531476,CCCCCCCCCC(=O)[O-],1.787584,CCCCCCCCCCN,0.381855,CCCCCCCCCC[N+](C)(C)C.[Br-],0.071390,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.475907,CCCCCCCCCCO,0.589175,CCCCCCCCCC(=O)OCC(CO)O,0.113329,2
4,CCCCCCCCCC(=O)O,1.170933,CCCCCCCCCC(=O)[O-],1.441019,CCCCCCCCCCN,1.677097,CCCCCCCCCC[N+](C)(C)C.[Br-],0.885832,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.011601,CCCCCCCCCCO,0.750236,CCCCCCCCCC(=O)OCC(CO)O,0.300105,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,CCCCCCCCCC(=O)O,0.112435,CCCCCCCCCC(=O)[O-],0.107957,CCCCCCCCCCN,0.009950,CCCCCCCCCC[N+](C)(C)C.[Br-],0.639219,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.020651,CCCCCCCCCCO,0.000000,CCCCCCCCCC(=O)OCC(CO)O,0.007968,0
332,CCCCCCCCCC(=O)O,0.019803,CCCCCCCCCC(=O)[O-],0.006976,CCCCCCCCCCN,0.064851,CCCCCCCCCC[N+](C)(C)C.[Br-],0.916291,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.821980,CCCCCCCCCCO,0.033918,CCCCCCCCCC(=O)OCC(CO)O,0.045929,59
333,CCCCCCCCCC(=O)O,0.137150,CCCCCCCCCC(=O)[O-],0.076961,CCCCCCCCCCN,0.043059,CCCCCCCCCC[N+](C)(C)C.[Br-],0.548121,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],1.090244,CCCCCCCCCCO,0.013410,CCCCCCCCCC(=O)OCC(CO)O,0.018822,2
334,CCCCCCCCCC(=O)O,0.013903,CCCCCCCCCC(=O)[O-],0.028587,CCCCCCCCCCN,0.092579,CCCCCCCCCC[N+](C)(C)C.[Br-],0.693147,CCCCCCCCCCOS(=O)(=O)[O-].[Na+],0.974560,CCCCCCCCCCO,0.049742,CCCCCCCCCC(=O)OCC(CO)O,0.036332,15


In [4]:
X_smi = get_latent_space(df_structured)
y_smi = df_structured['vesicles_formation']

Random Seed: 12345
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding
Using Rotation Embedding


/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Vocab size: 2393
[INFERENCE MODE - smi-ted-Light]


100%|██████████| 7/7 [00:01<00:00,  4.51it/s]
/Users/haoliu/Documents/GitHub/FoundationModel_on_Liposome/code/notebooks/../src/utils.py:80: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_train_emb = train_df.applymap(lambda x: replace_with_list(x, train_dict))


In [5]:
X_smi.shape

(336, 768)

In [6]:
df_total = pd.concat([X_smi, y_smi],axis=1)
df_total

/Users/haoliu/anaconda3/envs/ActiveLearning/lib/python3.11/site-packages/rdkit/Chem/PandasPatcher.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  for col in df_subset.columns[df_subset.applymap(MolFormatter.is_mol).any()]


,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,vesicles_formation
0,2.133225,-3.162936,0.789541,2.850490,3.396666,-4.821146,8.909615,0.670196,6.191031,1.689416,...,-1.362730,1.848185,2.106292,-3.763829,3.503884,-4.127567,6.484612,4.386611,-0.428854,22
1,1.466875,-2.214961,0.532058,1.976126,2.356294,-3.329292,6.124197,0.456357,4.332312,1.173333,...,-0.989388,1.304590,1.499760,-2.588206,2.457719,-2.849032,4.486022,3.019749,-0.259270,9
2,2.070614,-3.163397,0.776341,2.803670,3.371502,-4.708692,8.816238,0.659105,6.148667,1.676970,...,-1.333328,1.801240,2.095541,-3.742691,3.464535,-4.109267,6.426855,4.348367,-0.474272,6
3,1.751604,-2.508866,0.616528,2.436253,2.772160,-4.200623,7.392146,0.559019,5.247682,1.451382,...,-1.147136,1.592209,1.751879,-3.133081,3.034003,-3.411949,5.563547,3.599442,-0.132203,2
4,2.117789,-3.197670,0.769155,2.933910,3.431244,-5.015594,9.206407,0.678637,6.277452,1.818453,...,-1.279608,1.855526,2.025818,-3.943600,3.674249,-4.324271,6.849276,4.486199,-0.368311,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.589320,-0.836328,0.224420,0.760847,0.917404,-1.273870,2.338048,0.184664,1.734981,0.408111,...,-0.426882,0.516856,0.645956,-0.968425,0.894044,-1.038227,1.661256,1.172309,-0.127442,0
332,0.599943,-0.894980,0.231688,0.759187,0.954486,-1.251251,2.392987,0.183097,1.724623,0.419765,...,-0.429380,0.509492,0.633305,-0.988583,0.892973,-1.079294,1.670890,1.200181,-0.158670,59
333,0.596510,-0.830975,0.225542,0.773902,0.915194,-1.306595,2.368558,0.188725,1.762916,0.413704,...,-0.425143,0.527834,0.657649,-0.987289,0.911618,-1.048112,1.693009,1.188180,-0.130089,2
334,0.589221,-0.840813,0.225202,0.754606,0.913834,-1.264551,2.344799,0.183944,1.716336,0.409099,...,-0.416270,0.510532,0.634535,-0.975130,0.885118,-1.043602,1.657928,1.176208,-0.144483,15


In [7]:
df_total_cls = binarize_last_column(df_total)

In [8]:
df_total_cls.to_csv('df_total_amphiphiles_smi-ted_20250709.csv', index=False)